<a href="https://colab.research.google.com/github/Karthick47v2/mock-buddy/blob/main/face_landmark_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection
import tensorflow
from tensorflow import keras

In [3]:
y_train = np.load('/content/gdrive/MyDrive/data-v/y_train30.npy')
frames = np.load('/content/gdrive/MyDrive/data-v/frames30.npy')

In [4]:
img_size = 224

In [ ]:
# creating sequential model
model = keras.models.Sequential()

# using MobileNetV2 as feature extractor // include_top=False -> discarding last few layers and going to add custom layers for landmark detection
mn_v2 = keras.applications.MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')

In [6]:
model.add(mn_v2)
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(136))

model.build(input_shape=(None, 224, 224, 3))

In [ ]:
model.summary()

In [8]:
model.compile(optimizer='adam', loss="mean_squared_error", metrics=["mae", "accuracy"])

chk_path = "trained_model/model"
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=chk_path,
                                                   monitor="val_mae",
                                                   mode="auto",
                                                   save_best_only=True,
                                                   save_weights_only=True
                                                   )

In [9]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.9,
                                              monitor="val_mae",
                                              mode="auto",
                                              cooldown=0,
                                              patience=5,
                                              verbose=1,
                                              min_lr=1e-6
                                              )

In [10]:
train_frames, val_frames, train_lm, val_lm = sklearn.model_selection.train_test_split(frames, y_train, test_size=0.1, random_state=42)

In [11]:
EPOCHS=300
BATCH_SIZE=32

In [ ]:
history=model.fit(train_frames,
                  train_lm,
                  validation_data=(val_frames, val_lm),
                  batch_size=BATCH_SIZE,
                  epochs=EPOCHS,
                  callbacks=[model_checkpoint, reduce_lr]
                  )

Epoch 1/300
268/268 [==============================] - 70s 205ms/step - loss: 0.0198 - mae: 0.0889 - accuracy: 0.1000 - val_loss: 9.7530 - val_mae: 3.0380 - val_accuracy: 0.0933 - lr: 0.0010
Epoch 2/300
268/268 [==============================] - 55s 204ms/step - loss: 0.0053 - mae: 0.0567 - accuracy: 0.1587 - val_loss: 1.9718 - val_mae: 1.3525 - val_accuracy: 0.1449 - lr: 0.0010
Epoch 3/300
268/268 [==============================] - 56s 207ms/step - loss: 0.0043 - mae: 0.0513 - accuracy: 0.2136 - val_loss: 0.2960 - val_mae: 0.5179 - val_accuracy: 0.3771 - lr: 0.0010
Epoch 4/300
268/268 [==============================] - 56s 208ms/step - loss: 0.0040 - mae: 0.0494 - accuracy: 0.2656 - val_loss: 0.0868 - val_mae: 0.2768 - val_accuracy: 0.2844 - lr: 0.0010
Epoch 5/300
268/268 [==============================] - 55s 207ms/step - loss: 0.0039 - mae: 0.0484 - accuracy: 0.3117 - val_loss: 0.0468 - val_mae: 0.2014 - val_accuracy: 0.1732 - lr: 0.0010
Epoch 6/300
268/268 [========================

In [ ]:
# model_layers=[
#     keras.layers.Conv2D( 256, input_shape=( img_size , img_size , 3 ) , kernel_size=( 5 , 5 ) , strides=1 , activation='relu',name="input_layer"),
#     keras.layers.Conv2D( 256 , kernel_size=( 5 , 5 ) , strides=1 , activation='relu' ),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dropout(0.3),
    
#     keras.layers.Conv2D( 256, kernel_size=( 5 , 5 ) , strides=1 , activation='relu' ),
#     keras.layers.Conv2D( 256, kernel_size=( 5 , 5 ) , strides=1 , activation='relu' ),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dropout(0.3),
    
#     keras.layers.Conv2D( 200, kernel_size=( 5 , 5 ) , strides=2 , activation='relu'),
#     keras.layers.Conv2D( 200 , kernel_size=( 5 , 5 ) , strides= 1, activation='relu'),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dropout(0.3),
    
#     keras.layers.Conv2D( 200, kernel_size=( 5 , 5 ) , strides=1 , activation='relu'),
#     keras.layers.Conv2D( 200 , kernel_size=( 5 , 5 ) , strides=1 , activation='relu' ),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dropout(0.3),
    
#     keras.layers.Conv2D( 170, kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
#     keras.layers.Conv2D( 170, kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dropout(0.3),
    
#     keras.layers.Conv2D( 136, kernel_size=( 3 , 3 ) , strides=1 , activation='relu'),
#     keras.layers.Conv2D( 136, kernel_size=( 3 , 3 ) , strides=2 , activation='relu'),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dropout(0.3),
    
#     keras.layers.Conv2D( 136, kernel_size=( 3 , 3 ) , strides=2 , activation='relu'),
#     keras.layers.Conv2D( 136 , kernel_size=( 3 , 3 ) , strides=1 , activation='sigmoid'),

    
# ]
# model=keras.Sequential(model_layers)
# model.compile( loss= keras.losses.mean_squared_error , optimizer= keras.optimizers.Adam( lr=0.001 ) )
# model.summary()

In [ ]:
train = model.fit(frames, y_train, epochs=3, batch_size=32, verbose=2)

In [ ]:
model.save('/content/gdrive/MyDrive/data-v/mymodel')

In [ ]:
model = keras.models.load_model('/content/gdrive/MyDrive/data-v/mymodel')

In [ ]:
test_frames = np.load('/content/gdrive/MyDrive/data-v/test_frames.npy')

In [ ]:
fig, arr = plt.subplots(6,5,figsize=(20,20))

for i in range(1, 30):
  r = i // 5
  c = i % 5

  sam = test_frames[i]
  pred = model.predict( test_frames[ i : i +1  ] ) 
  x=np.reshape(pred[:,:,:,np.arange(0,136,2)],(68))*img_size
  y=np.reshape(pred[:,:,:,np.arange(1,136,2)],(68))*img_size
  arr[r][c].imshow(sam)
  arr[r][c].scatter( x,y, c='yellow',s=6)